In [1]:
import pandas as pd
import pickle

menwomen = ['Men','Women']
mwcsv = ['olympicsoccer24men.csv','olympicsoccer24women.csv']

mw = 0

In [2]:
# use pdf from olympics to get names and teams

import pandas as pd
import pickle



if True:
    # Step 1: Read the CSV file without headers
    df = pd.read_csvteamcsv(mwcsv[mw], header=None)
    
    # Step 2: Assign column names
    # Adjust based on the actual number of columns in the CSV
    df.columns = ['Team', 'Name']
    
    # Step 3: Drop rows with any missing or empty values in 'Team' or 'Name'
    df = df.dropna(subset=['Team', 'Name'])  # Remove rows with NaN values in 'Team' or 'Name'
    df = df[(df['Team'].str.strip() != '') & (df['Name'].str.strip() != '')]  # Remove rows with empty strings
    
    # Pickle the DataFrame
    pickle.dump(df, open('pkls/soccerdf.pkl', 'wb'))
    
else:
    
    # Load the pickled DataFrame
    df = pickle.load(open('pkls/soccerdf.pkl', 'rb'))
    
# Display the cleaned DataFrame
print(df)


AttributeError: module 'pandas' has no attribute 'read_csvteamcsv'

In [ ]:
# get wiki url for players

import pandas as pd
import requests
import time

if True:

    # Step 4: Function to get Wikipedia page URL
    def get_wikipedia_page_url(name):
        search_url = "https://en.wikipedia.org/w/api.php"
        params = {
            'action': 'query',
            'format': 'json',
            'list': 'search',
            'srsearch': name,
            'utf8': 1,
            'srlimit': 1
        }
        response = requests.get(search_url, params=params)
        data = response.json()
        
        if 'query' in data and 'search' in data['query']:
            search_results = data['query']['search']
            if search_results:
                page_title = search_results[0]['title']
                page_url = f"https://en.wikipedia.org/wiki/{page_title.replace(' ', '_')}"
                return page_url
        return None
    
    # Step 5: Add Wikipedia URLs to DataFrame
    df['Name_Link'] = df['Name'].apply(lambda name: get_wikipedia_page_url(name) or None)
    
    # Add delay to avoid rate limiting
    time.sleep(1)

    # Pickle the DataFrame
    pickle.dump(df, open('pkls/soccerdf.pkl', 'wb'))
    
else:
    
    # Load the pickled DataFrame
    df = pickle.load(open('pkls/soccerdf.pkl', 'rb'))

# Display the DataFrame with Wikipedia URLs
print(df)


In [ ]:
print(df['Name_Link'].isna().sum())

In [ ]:
# check if webpage is good if not make url=None

import pandas as pd

# Function to validate URLs
def validate_url(row):
    name = row['Name']
    url = row['Name_Link']
    
    if pd.isna(url):
        return None
    
    # Extract the part after the last '/'
    url_name_part = url.rsplit('/', 1)[-1]
    
    # Compare the first 3 letters
    if url_name_part[:3].lower() == name[:3].lower():
        return url
    else:
        return None

# Apply the validation function to each row
df['Name_Link'] = df.apply(validate_url, axis=1)

# Display the updated DataFrame
print(df)


In [ ]:
df['Name_Link'].isna().sum()

In [ ]:
# make sure web pages are ok

import pandas as pd

# Function to validate URLs
def validate_url(row):
    name = row['Name']
    url = row['Name_Link']
    
    if pd.isna(url):
        return None
    
    # Extract the part after the last '/'
    url_name_part = url.rsplit('/', 1)[-1]
    
    # Compare the first 3 letters
    if url_name_part[:3].lower() == name[:3].lower():
        return url
    else:
        return None

# Apply the validation function to each row
df['Name_Link'] = df.apply(validate_url, axis=1)

# Display the updated DataFrame
print(df)


In [ ]:
df.to_csv('cleaned_olympicsoccer.csv', index=False)

In [ ]:
# read in capitals

df_capitals = pd.read_csv('capitals.csv', encoding='latin1')
df_capitals.head()

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

if True:

    # Function to extract details from Wikipedia infobox
    def extract_infobox_details(url):
        if url is None:
            return None, None, None
        
        try:
            response = requests.get(url)
            soup = BeautifulSoup(response.content, 'html.parser')
            
            infobox = soup.find('table', {'class': 'infobox'})
            
            if infobox is None:
                return None, None, None
            
            # Extracting photo link
            img_tag = infobox.find('img')
            photo_link = 'https:' + img_tag['src'] if img_tag else None
            
            # Extracting birthplace and birthplace link
            birthplace_data = infobox.find('th', string='Place of birth')
            if birthplace_data is None:
                return None, None, photo_link  # No birthplace data, return photo link
            
            birthplace_data = birthplace_data.find_next_sibling('td')
            birthplace = birthplace_data.get_text(" ", strip=True)
            birthplace_links = birthplace_data.find_all('a')
            birthplace_link = ['https://en.wikipedia.org' + a['href'] for a in birthplace_links]
            
            return birthplace, birthplace_link, photo_link
        except Exception as e:
            return None, None, f'Error: {e}'
    
    # Apply the function to each URL in the DataFrame
    def process_row(row):
        birthplace, birthplace_link, photo_link = extract_infobox_details(row['Name_Link'])
        # Handle None for birthplace and links
        if birthplace is None:
            birthplace = None
        if birthplace_link is None:
            birthplace_link = None
        return pd.Series([birthplace, birthplace_link, photo_link], index=['birthplace', 'birthplace_link', 'photo_link'])
    
    # Apply function and add new columns to the DataFrame
    df[['birthplace', 'birthplace_link', 'photo_Link']] = df.apply(process_row, axis=1)

    # Pickle the DataFrame
    pickle.dump(df, open('pkls/soccerdf.pkl', 'wb'))
    
else:
    
    # Load the pickled DataFrame
    df = pickle.load(open('pkls/soccerdf.pkl', 'rb'))

# Set pandas options to display full content
pd.set_option('display.max_colwidth', None)  # Ensure full content is displayed

# Print the DataFrame to see all details
print(df)


In [ ]:
df.columns

In [ ]:
# get summary

import pandas as pd
import requests
import time

if True:

  # Function to fetch Wikipedia page summary
  def fetch_wikipedia_summary(link):
      time.sleep(0.2)  # Add a delay of 0.2 seconds between requests
      try:
          # Extract page title from the link
          title = link.split('/')[-1]
          url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{title}"
          response = requests.get(url)
          data = response.json()
          if 'extract' in data:
              return data['extract']
      except Exception as e:
          print(f"Error fetching summary for {link}: {e}")
      return None

  # Apply the function to each row in the DataFrame
  df['summary'] = df['Name_Link'].apply(fetch_wikipedia_summary)


  # Pickle the DataFrame
  pickle.dump(df, open('pkls/soccerdf.pkl', 'wb'))

else:

  # Load the pickled DataFrame
  df = pickle.load(open('pkls/soccerdf.pkl', 'rb'))

print(df)




In [ ]:
import pandas as pd
import requests
import pickle

if True: 
    # Function to fetch coordinates from Wikipedia API with error handling
    def fetch_coordinates_from_wikipedia(link):
        try:
            # Extract page title from the link
            title = link.split('/')[-1]
            url = f"https://en.wikipedia.org/w/api.php?action=query&prop=coordinates&format=json&titles={title}"
            response = requests.get(url)
            data = response.json()
            pages = data.get('query', {}).get('pages', {})
            if pages:
                page = next(iter(pages.values()))  # Get the first page (should be only one)
                coords_list = page.get('coordinates', [])
                if coords_list:
                    # Get the first item in the coordinates list
                    coords = coords_list[0]
                    return (coords.get('lat'), coords.get('lon'))
            # If there are no pages or coordinates, return None
            return None
        except Exception as e:
            # Uncomment for debugging
            # print(f"Error fetching coordinates for {link}: {e}")
            return None
    
    # Assuming df['birthplace_link'] contains lists of URLs
    def get_first_coords_from_list(row):
        if isinstance(row, list) and row:
            first_url = row[0]
            return fetch_coordinates_from_wikipedia(first_url)
        return None
    
    # Example DataFrame creation
    # df = pd.DataFrame({'birthplace_link': [['https://en.wikipedia.org/wiki/Some_City'], ['https://en.wikipedia.org/wiki/Another_City']]})
    
    # Apply the function to get coordinates for the first URL in the list
    df['birth_coords'] = df['birthplace_link'].apply(get_first_coords_from_list)

    # Pickle the DataFrame
    pickle.dump(df, open('pkls/soccerdf.pkl', 'wb'))

else:

    # Load the pickled DataFrame
    df = pickle.load(open('pkls/soccerdf.pkl', 'rb'))



print(df)


In [ ]:
# make capital in home not known

# Remove duplicate countries in df_capitals
df_capitals_unique = df_capitals.drop_duplicates(subset='Country')

# Create a mapping from Country to Coordinates
team_to_coords = df_capitals_unique.set_index('Country')[['Latitude', 'Longitude']].to_dict(orient='index')

# Function to fill missing coordinates
def fill_missing_coords(row):
    if row['birth_coords'] is None:
        team = row['Team']
        coords = team_to_coords.get(team)
        if coords:
            return (coords['Latitude'], coords['Longitude'])
    return row['birth_coords']

# Apply the function to fill missing coordinates
df['birth_coords'] = df.apply(fill_missing_coords, axis=1)

print(df)

In [ ]:
# save with correct gender

    # Pickle the DataFrame
    pickle.dump(df, open('pkls/soccer'+menwomen[mw]+'df.pkl', 'wb'))